# Import with Neo4j-admin

In [1]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
NAME_DB = "neo4j"

driver = GraphDatabase.driver(URI)

driver.verify_connectivity()
print("Successfully connected to Neo4j")

session = driver.session(database="system")

if [NAME_DB] in session.run("SHOW DATABASE yield name;").values():
    print("Database already exists")
else:
    session.run(f"CREATE DATABASE {NAME_DB};")
    print("Database created")
session = driver.session(database=NAME_DB)

Successfully connected to Neo4j
Database already exists


In [2]:
# CONSTRAINTS for unique IDs
session.run(
    """
    CREATE CONSTRAINT UniqueMovieID IF NOT EXISTS 
    FOR (m:Movie) 
    REQUIRE m.id IS unique;
    """
)
session.run(
    """
    CREATE CONSTRAINT UniqueUserID IF NOT EXISTS 
    FOR (u:User) 
    REQUIRE u.id IS unique;
    """
)
session.run(
    """
    CREATE CONSTRAINT UniqueGenreID IF NOT EXISTS 
    FOR (g:Genre) 
    REQUIRE g.id IS unique;
    """
)

In [ ]:
session.run(
    """
    CREATE INDEX RangeIndexTimestampRatings FOR ()-[r:RATED]-() ON (r.timestamp)
    """
)

## Follow this step on your shell

Start docker:
```
docker compose -d
```

Go to the container's shell

```
 docker exec -it neo4j_server /bin/bash
```

After that, you can use neo4j-admin command 
```
neo4j-admin database import full movies --overwrite-destination --id-type=integer --nodes import/users.csv import/movies.csv import/genres.csv --relationships import/ratings.csv import/movies_genres.csv
```

Restart docker container and you will see movies database with data.

After that the result output should be that:  
![alt text](image-1.png)

For consistency check you can run this command:
```
neo4j-admin database check neo4j
```